In [1]:
# Setup ipython environment
%load_ext autoreload
%autoreload 2
%matplotlib inline
# Setup the notebook's environment
import lalsimulation as lalsim
import lal
import numpy as np
from matplotlib.pyplot import *
from positive import *
from nrutils import lalphenom
from positive.units import *
from nrutils import scsearch,gwylm
from xcp import *

(positive)>> Applying custom matplotlib settings.
(init)>> Warm greetings from XCP.
(xcp.core)>> Catalog of calibration runs stored to "xcp.xcp_catalog"
(xcp.core)>> Metadata dictionary for Ed's catalog paper stored to "xcp.catalog_paper_metadata"


In [2]:
# Locate simulation                                                                                                                                                                                         
simlist = 'q8a08t120dP2_r03_T_96_360'

aa = scsearch( keyword=simlist, notkeyword=('80_Points','ASJmodified','.0.1.0'), verbose=True, unique=True )


# Calculate time domain phase derivative                                                                                                                                                                    
data = {}
for a in aa:

    # Load simulation                                                                                                                                                                                       
    y = gwylm( a, lmax=2, verbose=False, clean=True, load_dynamics=False )

    # Put into J-of-t frame                                                                                                                                                                                 
    yJ = y.__calc_j_of_t_frame__(enforce_initial_J_consistency=True)
#     yJ = y.__calc_final_j_frame__()

    # Put into co-precessing frame, symmetrise and clean                                                                                                                                                    
    ycptemp1 = yJ.__calc_coprecessing_frame__( transform_domain='td', kind='psi4' )
    ycptemp2 = ycptemp1.__symmetrize__()
    
    for lm in ycptemp2.lm:
        l,m = lm
        if (l,abs(m)) != (2,2):
            for kind in ycptemp2.lm[lm]:
                wfarr = ycptemp2[lm][kind].wfarr
                wfarr[:,1:] *= 1e-10
                ycptemp2[lm][kind].setfields( wfarr )

    alpha  = yJ.radiation_axis_info.radiation_axis['td_alpha']
    beta   = yJ.radiation_axis_info.radiation_axis['td_beta']
    gamma  = yJ.radiation_axis_info.radiation_axis['td_gamma']
    angles = ( alpha,beta,gamma )
    yJsym  = ycptemp2.__rotate_frame_at_all_times__( angles )

    # Calculate precession angles                                                                                                                                                                           
    ycp = yJsym.__calc_coprecessing_frame__( transform_domain='td', kind='psi4' )

(scsearch)>> Found keyword (=q8a08t120dP2_r03_T_96_360) keyword.
(scsearch)>> Found notkeyword (=('80_Points', 'ASJmodified', '.0.1.0')) keyword.
(scsearch)>> Found unique (=True) keyword.
(scsearch)>> Found verbose (=True) keyword.
(scsearch)>> List of keywords or string keyword found: ALL scentry objects matching will be passed. To pass ANY entries matching the keywords, input the keywords using an iterable of not of type list.
(scsearch)>> ## Found 1 unique simulations:
(scsearch)>> [0001][hogshead] q8precessing: 1chi0.80-p-q8.00	(q8a08t120dP2_r03_T_96_360_OLD)

(1.1485347585510402e-06, 1.1496084548900528e-06)
(romline!)>> N is 2, but it must be less than or equal to 3. We have set N=3. This course is often desired in cases where N=2 is naively given.
(__calc_radiated_linear_momentum__!)>> The current objects max ell value is 2, but we recommend a value of at least 4 for accurate determination of radiated linear momentum.
(__calc_radiated_linear_momentum__!)>> The current objects ma